This notebook `.ipynb` aims to do analysis on the `results/flores200_likelihood_cosine_similarity_results` result from the `experiments/likelihood_cosine_similarity_experiment` experiment.

In [2]:
import pandas as pd

In [3]:
result_df = pd.read_csv("../results/flores200_likelihood_cosine_similarity_results.csv")
result_df

,row_index,source_lang,source_script,source_text,target_lang,target_script,target_text,cosine_similarity
0,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",ace,Arab,"""کامو جينو نا تيكويه عمو ٤ بولن ڽڠ هانا ديابيت...",0.216057
1,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",ace,Latn,"""Kamöe jinoe na tikoih umu 4 buleuen nyang han...",0.214697
2,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",acm,Arab,"أضاف بكلامه وكَال، ""عدنا هسه فيران عمرها 4 أشه...",0.008297
3,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",acq,Arab,"وزاد قال، ""معنا الآن عكابر يصل عمرها 4 أشهر، و...",0.036192
4,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",aeb,Arab,وزاد قال، عندنا فيران عمرها 4 شهور كانت مريضة ...,0.115982
...,...,...,...,...,...,...,...,...
5095,24,eng,Latn,"Late on Sunday, the United States President Do...",yue,Hant,美國總統唐納德・特朗普於週日晚上透過新聞秘書發表聲明，宣布美國軍隊將撤離敘利亞。,0.404609
5096,24,eng,Latn,"Late on Sunday, the United States President Do...",zho,Hans,周日晚些时候，美国总统唐纳德·特朗普通过新闻秘书发表声明，宣布美军将撤离叙利亚。,0.385753
5097,24,eng,Latn,"Late on Sunday, the United States President Do...",zho,Hant,星期日傍晚，美國總統唐納‧川普透過新聞秘書發表聲明，宣布美國軍隊將撤離敘利亞。,0.514054
5098,24,eng,Latn,"Late on Sunday, the United States President Do...",zsm,Latn,"Lewat pada hari Ahad, Presiden Amerika Syarika...",0.036031


In [4]:
result_df.head()

,row_index,source_lang,source_script,source_text,target_lang,target_script,target_text,cosine_similarity
0,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",ace,Arab,"""کامو جينو نا تيكويه عمو ٤ بولن ڽڠ هانا ديابيت...",0.216057
1,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",ace,Latn,"""Kamöe jinoe na tikoih umu 4 buleuen nyang han...",0.214697
2,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",acm,Arab,"أضاف بكلامه وكَال، ""عدنا هسه فيران عمرها 4 أشه...",0.008297
3,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",acq,Arab,"وزاد قال، ""معنا الآن عكابر يصل عمرها 4 أشهر، و...",0.036192
4,0,eng,Latn,"""We now have 4-month-old mice that are non-dia...",aeb,Arab,وزاد قال، عندنا فيران عمرها 4 شهور كانت مريضة ...,0.115982


In [5]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   row_index          5100 non-null   int64  
 1   source_lang        5100 non-null   object 
 2   source_script      5100 non-null   object 
 3   source_text        5100 non-null   object 
 4   target_lang        5100 non-null   object 
 5   target_script      5100 non-null   object 
 6   target_text        5100 non-null   object 
 7   cosine_similarity  5100 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 318.9+ KB
